In [223]:
from astropy.io import fits
from astropy.table import Table
from astropy.nddata import NDDataArray, StdDevUncertainty
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import collections
%matplotlib inline

# Basic Steps
For a given spatial pixel $p = (x,y)$ with $N$ values of observed line ratios $R_N(p)$ with errors on the line ratios $E_N(p)$ in same units as $R_N$, we use predicted line ratios as function of $A_N(G_0,n)$, to compute differences $\delta_N$ across all $A$:
$${\delta_N^2(p,G_0,n)} = {\left[ R_N(p) - A_N(G_0,n)\over E_N(x)\right] }^2$$
 
The reduced chi-squared for spatial pixel $i$ is then given by

$$\chi^2(p,G_0,n) =  \sum_N {\delta_N^2(p,G_0,n) \over N-1}$$

and the best-fit value of $(G_0,n)$ is the minimum of $\chi^2$  The range of $(G_0,n)$ over which $A_N$ have been computed must be large enough to encompass physically reasonable values in the ISM, typically $10^{-1} < G_0 < 10^7$ in Habing units and $1 < n < 10^7$ cm$^{-3}$

For a spatial map, we do the above computation over all $p$ to produce a spatial maps of $\chi^2$ and best-fit spatial maps of $(G_0,n)$.



In [241]:
# maybe this is just better storing this info in a Table.  Then adding models is 
# just adding to the table
class ModelRatioData:
    def __init__(self,numerator, denominator, identifier,filename,metallicity=1):
        self._numerator = numerator
        self._denominator =denominator
        self._identifier = identifier
        self._filename = filename + "web"
        self._metallicity= metallicity
        
    @property
    def identifier(self): return self._identifier
   
    @property 
    def filename(self): return self._filename
    
    @property 
    def metallicity(self): return self._metallicity
    
    @property
    def isSolarMetallicity(self):
        return self._metallicity == 1

class Measurement(NDDataArray):
    def __init__(self,*args, **kwargs):
        self._identifier = kwargs.pop('identifier', None)
        super().__init__(*args, **kwargs)
        
    @property
    def flux(self):
        return self.data
    
    @property
    def id(self):
        return self._identifier
    
    @property
    def error(self):
        return self.uncertainty
    
  
    
    def add(self,other):
        z=super().add(other)
        z._identifier = self.id + '+' + other.id
        return z
   
    def subtract(self,other):
        z=super().subtract(other)
        z._identifier = self.id + '-' + other.id
        return z
    
    def multiply(self,other):
        z=super().multiply(other)
        z._identifier = self.id + '*' + other.id
        return z
        
    def divide(self,other):
        z=super().divide(other)
        z._identifier = self.id + '/' + other.id
        return z
    
    def __add__(self,other):
        z=self.add(other)
        return z
    def __sub__(self,other):      
        z=self.subtract(other)
        return z
    
    def __mul__(self,other):
        z=self.multiply(other)
        return z
    
    def __truediv__(self,other):
        z=self.divide(other)
        return z
    
class PDRutils:
    def __init__(self,models):
        if type(models) == str:
            self.initializeFromFile(models)
        else:
            self._models = models
        
    def initializeFromFile(self,filename):
        self._models=Table.read(filename,format="ascii.ipac")
        self._models.add_index("label")
        
    def find_pairs(self,m):
        """Return the list of model ratios labels for the given list of measurement IDs"""
        if not isinstance(m, collections.abc.Iterable) or isinstance(m, (str, bytes)) :
            raise Exception("m must be an array of strings")
            
        for q in itertools.product(m,m):
            s = q[0]+"/"+q[1]
            if s in self._models["label"]:
                yield(s)
    
    def find_files(self,m,ext="fits"):
        """Return the list of model ratio files for the given list of measurement IDs"""
        if not isinstance(m, collections.abc.Iterable) or isinstance(m, (str, bytes)):
            raise Exception("m must be an array of strings")
        for q in itertools.product(m,m):
            s = q[0]+"/"+q[1]
            if s in self._models["label"]:
                yield(self._models.loc[s]["filename"]+"."+ext)
    
    def ratiocount(self,m):
        """Return the number of model ratios found for the given list of measurement IDs"""
        # since find_files is a generator, we can't use len(), so do this sum.
        # See https://stackoverflow.com/questions/393053/length-of-generator-output
        return(sum(1 for _ in self.find_files(m)))
                
    def read_fits(self,m):
        d = "/n/lupus/mpound/WITS/Docs/pdrt/"
        self._fitsfiles = dict()
        for p in self.find_files(m):
            self._fitsfiles[p] = fits.open(d+p)
    
                

In [116]:
ratiodict = {
    "OI_145/OI_63"   : "oioi",
    "OI_145/CII_158" : "o145cii",
    "OI_63/CII_158"  : "oicp",
    "CII_158/CI_609" : "ciici609",
    "CI_370/CI_609"  : "cici",
    "CII_158/CO_10"  : "ciico",
    "CI_609/CO_10"   : "cico",
    "CI_609/CO_21"   : "cico21",
    "CI_609/CO_32"   : "cico32",
    "CI_609/CO_43"   : "cico43",
    "CI_609/CO_54"   : "cico54",
    "CI_609/CO_65"   : "cico65",
    "CO_21/CO_10"    : "co2110",
    "CO_32/CO_10"    : "co3210",
    "CO_32/CO_21"    : "co3221",
    "CO_43/CO_21"    : "co4321",
    "CO_65/CO_10"    : "co6510",
    "CO_65/CO_21"    : "co6521",
    "CO_65/CO_54"    : "co6554",
    "CO_76/CO_10"    : "co7610",
    "CO_76/CO_21"    : "co7621",
    "CO_76/CO_43"    : "co7643",
    "CO_76/CO_54"    : "co7654",
    "CO_76/CO_65"    : "co7665",
    "CO_87/CO_54"   : "co8754",
    "CO_87/CO_65"   : "co8765",
    "CO_98/CO_54"   : "co9854",
    "CO_98/CO_65"   : "co9865",
    "CO_109/CO_54"   : "co10954",
    "CO_109/CO_65"   : "co10965",
    "CO_1110/CO_54"   : "co111054",
    "CO_1110/CO_65"   : "co111065",
    "CO_1211/CO_54"   : "co121154",
    "CO_1211/CO_65"   : "co121165",
    "CO_1312/CO_54"   : "co131254",
    "CO_1312/CO_65"   : "co131265",
    "CO_1413/CO_54"   : "co141354",
    "CO_1413/CO_65"   : "co141365",
    "OI_63+CII_158/FIR"     : "fir",
    "OI_145+CII_158/FIR"  : "firoi145",
  "SIII_Z1/FEII_Z1"  : "siii35feii26z1",
    "SIII_Z3/FEII_Z3"  : "siii35feii26z3",
    "H200S1_Z1/H200S0_Z1" : "h200s1s0z1",
    "H200S1_Z3/H200S0_Z3" : "h200s1s0z3",
    "H200S2_Z1/H200S0_Z1" : "h200s2s0z1",
    "H200S2_Z3/H200S0_Z3" : "h200s2s0z3",
    "H200S2_Z1/H200S1_Z1" : "h200s2s1z1",
    "H200S2_Z3/H200S1_Z3" : "h200s2s1z3",
    "H200S3_Z1/H200S1_Z1" : "h200s3s1z1",
    "H200S3_Z3/H200S1_Z3" : "h200s3s1z3",
    "H200S1_Z1/SIII_Z1" : "h200s1siiiz1",
    "H200S1_Z3/SIII_Z3" : "h200s1siiiz3",
    "H200S2_Z1/SIII_Z1" : "h200s2siiiz1",
    "H200S2_Z3/SIII_Z3" : "h200s2siiiz3",
    "H264Q1_Z1/H210S1_Z1" : "h264q110s1z1",
    "H264Q1_Z3/H210S1_Z3" : "h264q110s1z3"
}
a = list()
b = list()
for r in ratiodict:
    nd = r.split("/")
    if ("Z3" in r):
        z=3
    else:
        z=1
    b.append((nd[0],nd[1],r,ratiodict[r]+"web",z))
    a.append(ModelRatioData(nd[0],nd[1],r,ratiodict[r],z))
t = Table(rows=b,names=("numerator","denominator","label","filename","z"))
t.add_index("label")
t.write("current_models.tab",format="ascii.ipac",overwrite=True)

df = t.to_pandas()

In [230]:
m1 = Measurement(data=30.,uncertainty = StdDevUncertainty(5.),identifier="OI_145")
m2 = Measurement(data=10.,uncertainty = StdDevUncertainty(10.),identifier="CI_370")
m3 = Measurement(data=500.,uncertainty = StdDevUncertainty(50.),identifier="CO_10")
m4 = Measurement(data=100.,uncertainty = StdDevUncertainty(10.),identifier="CII_158")
m = [m1.id,m2.id,m3.id,m4.id]

In [243]:
p = PDRutils("current_models.tab")
m = pd.Series([m1.id,m2.id,m3.id,m4.id])
for z in p.find_pairs(m):
    print(z)
    print(z.split('/'))
p.ratiocount(m)

OI_145/CII_158
['OI_145', 'CII_158']
CII_158/CO_10
['CII_158', 'CO_10']


2

In [234]:
p.read_fits(m)
print(p._fitsfiles)

{'o145ciiweb.fits': [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7f11086382b0>], 'ciicoweb.fits': [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7f11086a4ac8>]}


In [218]:
q=m1*m2 
print(((m1.flux*m2.flux)**2 * ((m1.uncertainty.array/m1.flux)**2 + (m2.uncertainty.array/m2.flux)**2))**0.5)
print(q,q.uncertainty,q.id)

304.13812651491094
300.0 StdDevUncertainty(304.13812651) OI_145 * CI_370


In [205]:
q.id

'OI_145 / CI_370'